In [34]:
import rasterio
import xarray as xr
import datetime
import numpy as np
from affine import Affine
from osgeo import gdal_array, osr, gdal

In [ ]:
import salem

In [380]:
out_path = '../geotiff/test3.tif'
path = f'/home/sagus/Development/temp/wrfout_A_d01_2020-02-06_18:00:00'

CBA_EXTENT_2 = [-68.92827622349982,
                -37.235026223499816,
                -59.22651553359684,
                -27.533265533596833]

CBA_EXTENT = [-68.91031,
                -37.408794,
                -56.489685,
                -27.518177]


CBA_EXTENT_3 = [-67.06237622349982,
                -35.515026223499816,
                -57.26651553359684,
                -27.693265533596833]

WRF_EXTENT = [-538001.0623448786,
                -538000.0000000792,
                537998.9376551214,
                537999.9999999208]

KM_PER_DEGREE = 111.32
RESOLUTION = 4

In [3]:
def getGeoT(extent, nlines, ncols):
    # Compute resolution based on data dimension
    resx = (extent[2] - extent[0]) / ncols
    resy = (extent[3] - extent[1]) / nlines
    return [extent[0], resx, 0, extent[3], 0, -resy]

In [ ]:
def corregir_wrfout(ruta_wrfout):
    """Fixes variables dimensions
        Parameters
        ----------
        ruta_wrfout: str
            route to the nc file to fix
        Returns
        -------
        rundate

    """
    xds = xr.open_dataset(ruta_wrfout)
    variables = ['XLAT', 'XLONG', 'XLAT_U', 'XLONG_U', 'XLAT_V', 'XLONG_V']
    if len(xds.coords['XLAT'].shape) > 2:
        for var in variables:
            xds.coords[var] = xds.coords[var].mean(axis=0)
    rundate = datetime.datetime.strptime(xds.START_DATE, '%Y-%m-%d_%H:%M:%S')

    return rundate, xds

In [281]:
def cambiar_projection(in_array):
    # WRF Spatial Reference System
    source_prj = osr.SpatialReference()
    source_prj.ImportFromProj4('+proj=lcc +lat_0=-32.500008 +lon_0=-62.7 +lat_1=-60 +lat_2=-30 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs')
    # Lat/lon WSG84 Spatial Reference System
    target_prj = osr.SpatialReference()
    target_prj.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')

    # se configura la matriz destino
    sizex = int(((CBA_EXTENT[2] - CBA_EXTENT[0]) * KM_PER_DEGREE) / RESOLUTION)
    sizey = int(((CBA_EXTENT[3] - CBA_EXTENT[1] ) * KM_PER_DEGREE) / RESOLUTION)
    
    out_array = np.ndarray(shape=(in_array[0].shape[0], sizey, sizex))
    
    for time in range(in_array.shape[0]):
        # se carga el array en gdal y se le asigna la projección y transofrmación
        raw = gdal_array.OpenArray(in_array[time, :, :])
        raw.SetProjection(source_prj.ExportToWkt())
        raw.SetGeoTransform(getGeoT(WRF_EXTENT, raw.RasterYSize, raw.RasterXSize))

        grid = gdal.GetDriverByName('MEM').Create("tmp_ras", sizex, sizey, 1, gdal.GDT_Float32)
        # Setup projection and geo-transformation
        grid.SetProjection(target_prj.ExportToWkt())
        grid.SetGeoTransform(getGeoT(CBA_EXTENT, grid.RasterYSize, grid.RasterXSize))

        # reprojectamos
        gdal.ReprojectImage(raw,
                            grid,
                            source_prj.ExportToWkt(),                   
                            target_prj.ExportToWkt(),
                            gdal.GRA_Average,
                            options=['NUM_THREADS=ALL_CPUS']
                           )
        
        out_array[time] = grid.ReadAsArray()
        
    return out_array

In [386]:
def cambiar_projection_ppn(in_array: np.ndarray):
    """Convert Grid to New Projection.
        Parameters
        ----------
        in_array

    """
    # WRF Spatial Reference System
    source_prj = osr.SpatialReference()
    source_prj.ImportFromProj4('+proj=lcc +lat_0=-32.500008 +lon_0=-62.7 +lat_1=-60 +lat_2=-30 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs')
    # Lat/lon WSG84 Spatial Reference System
    target_prj = osr.SpatialReference()
    target_prj.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')

    # se configura la matriz destino
    sizex = int(((CBA_EXTENT[2] - CBA_EXTENT[0]) * KM_PER_DEGREE) / RESOLUTION)
    sizey = int(((CBA_EXTENT[3] - CBA_EXTENT[1]) * KM_PER_DEGREE) / RESOLUTION)

    out_array = np.ndarray(shape=(len(in_array.keys()), sizey, sizex))

    for t in in_array.keys():
        # loar gdal array y se le asigna la projección y transofrmación
        raw = gdal_array.OpenArray(in_array[t])
        raw.SetProjection(source_prj.ExportToWkt())
        raw.SetGeoTransform(getGeoT(WRF_EXTENT,
                                    raw.RasterYSize,
                                    raw.RasterXSize))

        grid = gdal.GetDriverByName('MEM').Create("tmp_ras",
                                                  sizex, sizey, 1,
                                                  gdal.GDT_Float32)
        # Setup projection and geo-transformation
        grid.SetProjection(target_prj.ExportToWkt())
        grid.SetGeoTransform(getGeoT(CBA_EXTENT,
                                     grid.RasterYSize,
                                     grid.RasterXSize))

        # reprojectamos
        gdal.ReprojectImage(raw,
                            grid,
                            source_prj.ExportToWkt(),
                            target_prj.ExportToWkt(),
                            gdal.GRA_Average,
                            options=['NUM_THREADS=ALL_CPUS'])

        out_array[t] = grid.ReadAsArray()

    return_list = []
    return_list.append(out_array)
    return_list.append(grid.GetGeoTransform())
    return_list.append(grid.GetProjection())

    return return_list

In [336]:
def guardar_tif(geoTransform: list, arr: np.ndarray, out_path: str):
    nw_ds = rasterio.open(out_path, 'w', driver='GTiff',
                          height=arr.shape[0],
                          width=arr.shape[1],
                          count=1, dtype=str(arr.dtype),
                          crs=target_prj.ExportToPrettyWkt(),
                          transform=Affine.from_gdal(*geoTransform))
    nw_ds.write(np.flipud(arr), 1)
    nw_ds.close()

In [337]:
# Open the NetCDF file
rundate, xds = corregir_wrfout(path)

NameError: name 'corregir_wrfout' is not defined

In [381]:
xds = xr.open_dataset(path)

In [206]:
xds.variables['T2'].values = xds.variables['T2'].values - 273.15
in_array = xds.variables['T2'].values[::-1, :]

In [ ]:
arrs = {}
for t in range(out_RAINNC.shape[0]):
    arrs[t] = out_RAINNC[t, :, :] + out_RAINC[t, :, :]
    arrs[t][arrs[t] == 0] = np.nan

In [382]:
arrs = {}
for t in range(len(xds.variables['Times'])):
    arrs[t] = xds.variables['RAINNC'].values[t, :, :] + xds.variables['RAINC'].values[t, :, : ]

In [332]:
arrs.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57])

In [387]:
return_list = cambiar_projection_ppn(arrs)

In [397]:
a = return_list.remove(1)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [300]:
out_ppn.shape

(58, 217, 272)

In [341]:
out_path = '../geotiff/ppn'
gtiff_id_list = []
for t in range(1, out_ppn.shape[0]):
    gtiff_id_list.append(guardar_tif(geoTransform, out_ppn[t, :, :] - out_ppn[t - 1, :, :], f"{out_path}_{t}.tif"))

In [342]:
gtiff_id_list.append(guardar_tif(geoTransform, out_ppn[33] - out_ppn[9], f"{out_path}.tif"))
gtiff_id_list.append(guardar_tif(geoTransform, out_ppn[45] - out_ppn[9], f"{out_path}_a36.tif"))
gtiff_id_list.append(guardar_tif(geoTransform, out_ppn[57] - out_ppn[9], f"{out_path}_a48.tif"))

In [317]:
out_ppn[33]

array([[1.69496692e-04, 9.15985147e-05, 6.85016721e-06, ...,
        1.22133617e+01, 4.20018768e+00, 0.00000000e+00],
       [2.24573514e-03, 2.24573514e-03, 7.69839913e-04, ...,
        1.04110098e+01, 3.67334414e+00, 0.00000000e+00],
       [2.89291912e-03, 1.81998918e-03, 1.27853383e-03, ...,
        9.65796661e+00, 3.39446020e+00, 0.00000000e+00],
       ...,
       [6.54956055e+01, 7.56948395e+01, 7.61632080e+01, ...,
        1.05218822e-03, 0.00000000e+00, 0.00000000e+00],
       [9.62429504e+01, 9.42198792e+01, 8.37908020e+01, ...,
        2.73825834e-03, 3.53475684e-06, 0.00000000e+00],
       [9.62156372e+01, 7.77455292e+01, 6.04714088e+01, ...,
        2.71705003e-03, 2.26743923e-05, 0.00000000e+00]])

In [318]:
out_ppn[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [178]:
out_path = '../geotiff/T2'
gtiff_id_list = []
for t in range(1, out_array.shape[0]):
    gtiff_id_list.append(guardar_tif(geoTransform, out_array[t, :, :], f"{out_path}_{t}.tif"))

In [242]:
arrs = {}
for t in range(out_RAINNC.shape[0]):
    arrs[t] = out_RAINNC[t, :, :] + out_RAINC[t, :, :]
    arrs[t][arrs[t] == 0] = np.nan

In [184]:
out_path = '../geotiff/ppn'
gtiff_id_list = []
for t in range(1, out_array.shape[0]):
    gtiff_id_list.append(guardar_tif(geoTransform, arrs[t] - arrs[t - 1], f"{out_path}_{t}.tif"))

In [187]:
gtiff_id_list.append(guardar_tif(geoTransform, arrs[33] - arrs[9], f"{out_path}.tif"))
gtiff_id_list.append(guardar_tif(geoTransform, arrs[45] - arrs[9], f"{out_path}_a36.tif"))
gtiff_id_list.append(guardar_tif(geoTransform, arrs[57] - arrs[9], f"{out_path}_a48.tif"))

In [243]:
a = arrs[57] - arrs[9]

In [251]:
arrs.values()

dict_values([array([[2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       ...,
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.]]), array([[2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       ...,
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.]]), array([[2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       ...,
       [2000., 2000., 2000., ..., 2000., 2000., 2000.],
       [2000., 2000., 2000., ..., 2000., 2000., 200

In [127]:
geoTransform = getGeoT(CBA_EXTENT, grid.RasterYSize, grid.RasterXSize)

In [323]:
geoProjection

[-67.06237622349983,
 0.03601419371287863,
 0,
 -27.693265533596833,
 0,
 -0.03604498013780177]

In [326]:
type(grid.GetProjection())

str

In [6]:
in_array = xds.variables['T2'][12].values[::-1, :]
raw = gdal_array.OpenArray(in_array)

In [65]:
# Setup projection and geo-transformation
raw.SetProjection(source_prj.ExportToWkt())
raw.SetGeoTransform(getGeoT(WRF_EXTENT, raw.RasterYSize, raw.RasterXSize))

0

In [126]:
in_array.dtype

dtype('float32')

In [67]:
sizex = int(((CBA_EXTENT[2] - CBA_EXTENT[0]) * KM_PER_DEGREE) / RESOLUTION)
sizey = int(((CBA_EXTENT[3] - CBA_EXTENT[1] ) * KM_PER_DEGREE) / RESOLUTION)

In [129]:
grid.GetProjection()

'GEOGCS["unknown",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]'

In [130]:
target_prj.ExportToPrettyWkt()

'GEOGCS["unknown",\n    DATUM["WGS_1984",\n        SPHEROID["WGS 84",6378137,298.257223563,\n            AUTHORITY["EPSG","7030"]],\n        AUTHORITY["EPSG","6326"]],\n    PRIMEM["Greenwich",0,\n        AUTHORITY["EPSG","8901"]],\n    UNIT["degree",0.0174532925199433,\n        AUTHORITY["EPSG","9122"]],\n    AXIS["Longitude",EAST],\n    AXIS["Latitude",NORTH]]'

In [69]:
grid = gdal.GetDriverByName('MEM').Create("tmp_ras", sizex, sizey, 1, gdal.GDT_Float32)

In [56]:
grid

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f58f8331c60> >

In [104]:
out_array = np.ndarray(shape=(in_array.shape[0], sizey, sizex))

In [105]:
out_array[0] = grid.ReadAsArray()

In [71]:
gdal.ReprojectImage(raw,
                    grid,
                    source_prj.ExportToWkt(),                   
                    target_prj.ExportToWkt(),
                    gdal.GRA_NearestNeighbour,
                    options=['NUM_THREADS=ALL_CPUS']
                   )

0

In [72]:
# Read grid data
array1 = grid.ReadAsArray()

In [75]:
grid

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f58f65a85d0> >

In [73]:
array1

array([[22.899048, 22.909882, 22.682526, ..., 23.672058, 23.696442,
         0.      ],
       [23.21399 , 22.955841, 22.828308, ..., 23.894104, 23.730713,
         0.      ],
       [23.57373 , 23.222015, 22.886871, ..., 23.959167, 23.726654,
        23.726654],
       ...,
       [20.962646, 20.962646, 21.002045, ..., 26.296814, 26.296814,
        26.652832],
       [20.958527, 20.958527, 20.975433, ..., 26.732697, 26.732697,
        26.588379],
       [20.932373, 20.932373, 20.952454, ..., 26.850403, 26.698608,
        26.522095]], dtype=float32)

In [74]:
nw_ds = rasterio.open(out_path, 'w', driver='GTiff',
                      height=grid.RasterYSize,
                      width=grid.RasterXSize,
                      count=1, dtype=np.float32,
                      crs=grid.GetProjection(),
                      transform=Affine.from_gdal(*grid.GetGeoTransform()))
nw_ds.write(grid.ReadAsArray(), 1)
nw_ds.close()

## Reproject

In [379]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
trans_cba = getGeoT(CBA_EXTENT, 269, 269)

In [ ]:
trans_cba

In [ ]:
with rasterio.open('../geotiff/test.tiff') as src:
    transform, width, height = calculate_default_transform(src.crs, target_prj.ExportToWkt(), src.width, src.height, *src.bounds)
    
    kwargs = src.meta.copy()

    kwargs.update({
        'crs': target_prj.ExportToWkt(),
        'transform': Affine.from_gdal(*trans_cba),
        'width': width,
        'height': height
    })
    with rasterio.open('../geotiff/test.wgs84.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=Affine.from_gdal(*trans_cba),
                dst_crs=target_prj.ExportToWkt(),
                resampling=Resampling.nearest)

In [ ]:
dst.bounds

In [ ]:
src.bounds

In [ ]:
transform

In [ ]:
Affine.from_gdal(*trans_cba)

## Load into dataframe

In [374]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
COLUM_REPLACE = {'Subcuenca': 'subcuenca', 'Cuenca': 'cuenca'}

In [345]:
def integrar_en_cuencas(cuencas_shp: str) -> gpd.GeoDataFrame:
    """
    This functions opens a geotiff with ppn data, converts to a raster,
    integrate the ppn into cuencas and returns a GeoDataFrame object.

    Parameters:
        cuencas_shp: Path to shapefile
    Returns:
        cuencas_gdf_ppn (GeoDataFrame): a geodataframe with cuerncas and ppn
    """
    cuencas_gdf: gpd.GeoDataFrame = gpd.read_file(cuencas_shp)
    df_zonal_stats = pd.DataFrame(zonal_stats(cuencas_shp, "geotiff/ppn.tif"))
    #df_zonal_stats = pd.DataFrame(zonal_stats(cuencas_shp, "geotiff/ppn_36.tif"))
    #df_zonal_stats = pd.DataFrame(zonal_stats(cuencas_shp, "geotiff/ppn_48.tif"))

    cuencas_gdf_ppn = pd.concat([cuencas_gdf, df_zonal_stats], axis=1).dropna(subset=['mean'])
    cuencas_gdf_ppn = cuencas_gdf_ppn.rename(columns=COLUM_REPLACE)
    return cuencas_gdf_ppn[['subcuenca', 'cuenca', 'geometry', 'count', 'max', 'mean', 'min']]

In [352]:
cuencas_shp = '../shapefiles/cuencas_hidro_new.shp'

In [ ]:
cuencas_gdf_ppn = integrar_en_cuencas('../shapefiles/cuencas_hidro_new.shp')


In [362]:
cuencas_gdf = gpd.read_file(cuencas_shp)


In [354]:
df_zonal_stats = pd.DataFrame(zonal_stats(cuencas_shp, "../geotiff/ppn.tif"))

/home/sagus/anaconda3/envs/wrfcartopy/lib/python3.8/site-packages/rasterstats/io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [364]:
df_zonal_stats_36 = pd.DataFrame(zonal_stats(cuencas_shp, "../geotiff/ppn_a36.tif"))

/home/sagus/anaconda3/envs/wrfcartopy/lib/python3.8/site-packages/rasterstats/io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [368]:
df_zonal_stats_36 = df_zonal_stats_36.rename(columns={"mean": "mean36"})

In [370]:
df_zonal_stats_48 = pd.DataFrame(zonal_stats(cuencas_shp, "../geotiff/ppn_a48.tif"))

In [378]:
for hour in ('', '36', '48'):
    print(f'main{hour}/')

main/
main36/
main48/


In [358]:
df_zonal_stats_48 = pd.DataFrame(zonal_stats(cuencas_shp, "../geotiff/ppn_a48.tif"))

In [ ]:
df_zonal_stats_48 = df_zonal_stats_48.rename(columns={"mean": "mean48"})

In [375]:
cuencas_gdf_ppn = pd.concat([cuencas_gdf, df_zonal_stats, df_zonal_stats_36['mean36'], df_zonal_stats_48['mean48']], axis=1).dropna(subset=['mean'])
cuencas_gdf_ppn = cuencas_gdf_ppn.rename(columns=COLUM_REPLACE)

In [376]:
cuencas_gdf_ppn

,LAYER,Codigo,subcuenca,Nombre_Sub,cuenca,CLOSED,BORDER_STY,BORDER_COL,BORDER_WID,FILL_STYLE,geometry,min,max,mean,count,mean36,mean48
0,Unknown Area Type,01,01-Sistema San Francisco,Sistema San Francisco,Río Pasaje o Salado,YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-62.35514 -31.44984, -62.35400 -31.4...",0.990576,49.935860,16.426509,46,16.870316,28.010894
1,Unknown Area Type,02,02-Cuenca R. Quinto (Popopis),Cuenca R. Quinto (Popopis),Río Quinto (Popopis),YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-65.11131 -33.91312, -65.11132 -33.9...",0.000000,0.000000,0.000000,115,0.000000,0.000000
2,Unknown Area Type,03,03-Sistema A. La Punilla,Cuenca R. Quinto (Popopis),Río Quinto (Popopis),YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-65.14244 -33.19465, -65.14217 -33.1...",0.000000,0.000000,0.000000,9,0.000000,0.000002
3,Unknown Area Type,04,04-Cuenca R. Anizacate,Cuenca R. Anizacate,Lag. Mar Chiquita,YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-64.87730 -31.63691, -64.87559 -31.6...",0.041645,3.821408,1.054001,84,2.140647,5.771755
4,Unknown Area Type,05,05-Cuenca A. La Cañada,Cuenca A. La Cañada,Lag. Mar Chiquita,YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-64.50765 -31.48776, -64.50737 -31.4...",0.059763,3.976195,1.048634,22,4.332008,6.635379
5,Unknown Area Type,06,06-Cuenca R. Saldán,Cuenca R. Saldán,Lag. Mar Chiquita,YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-64.43029 -31.13925, -64.43001 -31.1...",0.590173,1.777915,1.358020,16,7.346383,29.738773
6,Unknown Area Type,07,07-Cuenca Emb Los Molinos,Cuenca Emb Los Molinos,Lag. Mar Chiquita,YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-64.93432 -31.93632, -64.93431 -31.9...",0.000903,2.491025,0.405180,67,0.526212,11.292206
7,Unknown Area Type,09,09-Cuenca Emb. Piedras Moras,Cuenca Emb. Piedras Moras,Río Carcarañá,YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-64.96101 -32.19466, -64.96072 -32.1...",0.000000,3.355357,0.108769,284,0.630770,5.070171
8,Unknown Area Type,10,10-Sistema Laguna del 7,Área sin drenaje superficial,Río Quinto (Popopis),YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-64.22198 -34.23732, -64.22168 -34.2...",0.000000,0.000000,0.000000,70,0.000000,0.000000
9,Unknown Area Type,12,12-Sistema Bañados del R. Dulce,Sistema Bañados del R. Dulce,Lag. Mar Chiquita,YES,Solid,"RGB(0,0,0)",1,No Fill,"POLYGON ((-63.38420 -29.98726, -63.38245 -29.9...",5.895084,79.321088,27.450917,325,30.332509,37.210613
